# Project - NYT bestseller Search & Quiz

«NYT Bestseller Search & Quiz» is a project created in the context of the course “Programming with advanced computer languages” by Dr. Mario Silic in the spring semester 2021 at the University St. Gallen. The program allows users to search and filter the NYT bestsellers database and to test his knowledge in a quiz on bestseller books. The project used the Python programming language and dataset used was on the «New York Times Bestseller list» between 2010 and 2019, which is available on Kaggle for free: (https://www.kaggle.com/dhruvildave/new-york-times-best-sellers).

# 00 Initialization

In [1]:
# load all libraries needed for this script
import numpy as np
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
import random

In [2]:
### if need to install pacakges use, uncomment respective line
#%pip install python-dateutil
#%pip install pandas
#%pip install numpy
#%pip install datetime

# 01 Data
Loading & formating

In [3]:
# load clean data file
file = open('data/NYTB_clean.csv') # add: encoding = 'utf-8' if problem to read data

# empty list to append all data
data = []
# create one large list with many lists embedded (representing each row of the dataset)
for line in file:
    line_list = line.split(';')
    data.append(line_list)

# first row (or list) of data frame contain header
header = data[0]
print('Headers: ',header)

# NYTB data consists of the whole list except the first one with headers
data = data[1:-1]

# convert published date, price and week on NYT bestseller to date, integer and float
for line in data:
    line[1] = dt.datetime.strptime(line[1],'%Y-%m-%d')
    line[4] = float(line[4])
    line[5] = int(line[5])

# here select a smaller sample of 'data' if wanted using [x:y]
NYT_bestsellers = data

Headers:  ['title', 'published_date', 'author', 'description', 'price', 'weeks_on_list\n']


# 02 Basic Functions
Define function for search engine by criterion & function to display serach results

In [4]:
##### Function used to display results in a more visually pleasant way
### Display in a nice format output of function any search functions with "top" as argument for the number of books displayed
def res_search(lst, top = 5):
    # if filtered list is empty then print message
    if lst == []: print("Sorry, no book fits your search criteria.")
    
    # otherwise print each book in the list up to argument "top"
    else:
        i = 0
        #'\033[1m' + 'Question '+ str(question+1) +' out of ', str(rounds) + '\033[0m'
        print('\n\033[1mResults of your search:\033[0m')
        # display the title, author name(s) and release date of the first books in the list 
        for book in lst:
            print("-",book[0].title(),"by",book[2],"| First time on NYTB list:",book[1].date(),"| Weeks on NYTB list:",book[5],"| price ($):", book[4])
            i += 1
            # after x books, only display the length of the list --> avoid that the output is too long. (top-1 because index in python starts with 0)
            if i > (top-1): 
                print("\nOnly the first", top, "books have been displayed out of",len(lst),"books.")
                break

### function that display more books if user wants, argument needed: the result list of any search
def display_n_book(res_lst):
    # only if the result of the search has more than 5 books, we ask if they want to see more books
    if len(res_lst) > 5:
        # prompt user for an integer and if integer display the the number of books wished
        try: 
            n = int(input("Do you want to see more book of your search results? (currently 5)\nIf yes, type the number of books you want to see! If no, type anything else:"))
            n = np.minimum(n, len(res_lst))
            res_search(res_lst, n)
        # otherwise input is not an integer display nothing
        except ValueError: None
    else: None
        
### basic function to ask a binary question (answered by yes or no)
def binary_Q(question):
    # ask a question based on string "question" and add possible answer
    reply = input(question +' (yes/no): ').lower()
    
    # return true if answer is yes
    if reply == 'yes': return True
    # return false if answer is no
    elif reply == 'no': return False
    # ask quesiton again if answer is not yes or no
    else: return binary_Q('Input invalid. '+ question)

In [5]:
#### SEARCH by Title #####
### search by title function
def search_by_title(lst = NYT_bestsellers):
    print("Please enter the book title you want to search for.")
    # title converts in capital letter to fit the list format
    search = input("Title: ").upper()
    # empty list for results of the search
    results = [] 
    for book in lst:
        # book[0] denotes the title in the list
        if search in book[0]:
            # if element is in list columns append it to the variable "results"
            results.append(book)
    return results

In [6]:
#### SEARCH by Author #####
### search by author function
def search_by_author(lst = NYT_bestsellers):
    print("Please enter the author name(s) you want to search for.")
    # prompt user for an author and convert the string in cs
    search = input("Author(s): ").lower()
    # empty list for results of the search
    results = [] 
    for book in lst:
        # book[2] denotes the author name(s) in the list
        if search in book[2].lower():
            # if element is in list columns append it to the variable "results"
            results.append(book)
    return results

In [7]:
#### SEARCH by Date #####
### Function: transform a string into a date format
def string_to_date(string, start = 1):
    # convert string to the right format, raise ValueError if cannot
    if start ==1:
        if len(string) == 4: date = dt.datetime.strptime(string,'%Y')
        elif len(string) == 7: date = dt.datetime.strptime(string,'%Y-%m') 
        else: date = dt.datetime.strptime(string,'%Y-%m-%d')
    # convert string to the right format + add a time delta of one unit for later search, raise ValueError if cannot
    else: 
        if len(string) == 4: date = dt.datetime.strptime(string,'%Y') + relativedelta(years=1)
        elif len(string) == 7: date = dt.datetime.strptime(string,'%Y-%m') + relativedelta(months=1)
        else: date = dt.datetime.strptime(string,'%Y-%m-%d') + relativedelta(days=1)
    return date

### Function: prompt user to enter a date and check if fit format otherwise ask again
def prompt_date(string, start = 1):
    # if start = 1, then prompt user for starting date
    if start == 1:
        # use try and except to catch ValueError and prompt user again
        try:
            string2 = input('%s'%string)
            # use function string_to_date to convert the string to date format
            date = string_to_date(string2,1)
        # if ValueError promt user again
        except ValueError:
            print("The entered date does not fit the format required, try again!")
            return prompt_date(string,1)
    else: 
        # use try and except to catch ValueError and prompt user again
        try:
            string2 = input('%s'%string)
            # use function string_to_date to convert the string to date format and add one unit of time
            date = string_to_date(string2,0)
        # if ValueError promt user again
        except ValueError: 
            print("The entered date does not fit the format required, try again!")
            return prompt_date(string,0)
    return date

### search by date function, call two previous functions 
def search_by_date(lst = NYT_bestsellers):
    print("NYT bestsellers list contains information between 2010 and 2019 and is released every Wednesday")
    print("Please enter a start date and an end with once of the following format: YYYY-MM-DD, YYYY-MM or YYYY")
    print("If you want to look for one exact date, enter twice the same date.")
    # use built function to prompt user
    start = prompt_date("Starting date: ",1)
    end = prompt_date("Ending date: ",0)
    
    # use while loop in order to get an ending date that is after the starting date
    while start >= end:
        print('Ending date is before starting date, please enter a later date.')
        end = prompt_date("Ending date: ",0)
        
    # empty list for results of the search
    results = [] 
    for book in lst:
        # book[1] denotes the release date in the list
        if book[1] >= start and book[1] < end:  
            # if element is in list columns append it to the variable "results"
            results.append(book)
    return results

In [8]:
#### SEARCH by number of weeks on NYT bestseller list #####
# check if input is an integer
def prompt_int(string):
    # try to convert string to integer
    try:
        integer = int(input('%s'%string))
        return integer
    # if cannot, then prompt user again with warning message
    except ValueError:
        print("The value entered was not an integer, try again.")
        return prompt_int(string)

### search by number of week on NYT bestseller list
def search_by_week(lst = NYT_bestsellers):
    print("Please enter the minimum number of weeks a book has to be on the NYT bestsellers list:")
    print("If you want to display all books, enter 0.")
    # prompt user using the built funciton "prompt_int"
    search = prompt_int('Please, enter a number of weeks: ')
    
    # empty list for results of the search
    results = [] 
    for book in lst:
        # book[5] denotes the number of weeks on NYT bestseller list
        if book[5] >= search:
            # if element is in list columns append it to the variable "results"
            results.append(book)
    return results

In [9]:
#### SEARCH by book price #####
# check if input is an float, argument is the text to print when asking user for a float
def prompt_float(string):
    # try to convert input to a float
    try:
        number = float(input('%s'%string))
        return number
    # if cannot, raise error and prompt user again
    except ValueError:
        print("The value entered was not an integer, try again.")
        return prompt_float(string)

### Search by price function
def search_by_price(lst = NYT_bestsellers):
    print("Please enter a range for book price on the NYT bestseller list")
    print("Enter two prices, the order of two prices does not matter!")
    # prompt user for a float and check it using the built function "prompt_float"
    p1 = prompt_float('Price 1: ')
    p2 = prompt_float('Price 2: ')
    p_range = np.sort(np.array([p1, p2])) # first element is the smallest
    
    # empty list for results of the search
    results = [] 
    # book[4] denotes the price of a book in the list
    for book in lst:
        if book[4] >= p_range[0] and book[4] <= p_range[1]:
            # if element is in list columns append it to the variable "results"
            results.append(book)
    return results

### Check functions
Uncomment code line to check a specific function

In [10]:
#res_search(search_by_author())
#res_search(search_by_date())
#res_search(search_by_week())
#res_search(search_by_price())

# 03 Advanced Search Functions
Define search selection type function & advanced search function

In [11]:
### Define explicative text to be used in adv_search function
text1 = """It allows you to filter the dataset by multiple criteria. After one search, you can either search 
the filtered list again by another criterion or search the original dataset by another criterion."""

text2 = """\nPossible search criteria:
- book title                                (type: title)
- author name(s)                            (type: author)
- published date of the NYT bestseller list (type: date)
- number of weeks on the NYTB list          (type: week)
- book price range                          (type: price)"""


### prompt user to search by a specific criterion and call previously-defined functions
def search_select(lst = NYT_bestsellers, txt = text2):
    # prompt user for the search criterion
    print(txt)
    crit = input('Which criterion do you want to search by? ').lower()
    
    # check if crit is one of the possibilities
    if crit in ['title', 'author', 'date', 'week', 'price']:
        # if crit is title, then run search_by_title function and save result as filt_lst 
        if   crit == 'title': filt_lst = search_by_title(lst)
            
        # if crit is title, then run search_by_title function and save result as filt_lst 
        elif crit == 'author': filt_lst = search_by_author(lst)
            
        # if crit is title, then run search_by_title function and save result as filt_lst 
        elif crit == 'date': filt_lst = search_by_date(lst)
            
        # if crit is title, then run search_by_week function and save result as filt_lst 
        elif crit == 'week': filt_lst = search_by_week(lst)
            
        # if crit is price, then run search_by_price function and save result as filt_lst 
        else: filt_lst = search_by_price(lst)        
        
        return filt_lst
    
    # if user want to quit funciton, then return an empty list that will stop the function
    elif crit == 'quit':
        empty_lst = []
        return empty_lst
    
    # if input does not match one of the above, then prompt user again
    else:
        print('Input does not fit the requirements, or type: "quit" to quit the search.')
        # ask again for criterion without whole description
        return search_select(lst, txt = '') 

### Advanced Search function, that call previous function and enable to filter the NYT dataset multiple times
def adv_search():
    # greet user and introduce her/him to the function
    print('\n\033[1m' + 'Welcome to the advanced search!' + '\033[0m')
    # print text1 that explain the advanced search
    print(text1)
    
    # for the first prompt, we define the filtered list (filt_lst) as the NYT bestseller list.
    filt_lst = NYT_bestsellers
    answer = True
    
    # using a while loop to keep filter the existing list
    while answer:
        # rewrite filt_lst with the previous filt_lst filtered by another criterion
        filt_lst = search_select(filt_lst)
        
        # print current filtered list
        res_search(filt_lst)
        display_n_book(filt_lst)
        
        # if the filtered list has one or no book, we stop prompting the user
        if len(filt_lst) < 2:
            break
        # if the filtered list has more than one book, we ask user if she/he wants to continue
        else:
            # ask if user wants to continue search existing list by another criterion
            answer = binary_Q('Do you want to filter your search results by another criterion? ')
            
        

### Check functions
Uncomment code line to check a function

In [12]:
#search_select()
#adv_search()

# 04 Quiz function
Define quiz function and other function to run quiz

In [13]:
### Function that compute the score of player depending on the correctness of your answer
def new_score(options, answer, solution, score, coef = 1):
    # dictionary to convert quiz answer to integer
    d_ans = {"A": 0,  "B": 1, "C": 2, "D": 3}
    
    try:
        # If answer correct, 100 x coefficient (default = 1) points are awared to player
        if options[d_ans[answer]] == solution:
            score += 100*coef
            print("Correct answer! Your score is now", score)
            
        # If wrong answer selected, the user loses 30 points or nothing if he/she has 0 pts already
        else:
            # use np.minimum() to avoid that use get a negative score
            score -= np.minimum(30, score)
            print("Wrong answer! The correct answer was:",(str(solution)+"."),"Your score is now", score)
            
        return score 
    except Exception as e: print(e) 

In [14]:
### Function responsible for asking question and analysing answer, arguments used: 
# - lst = list of books (often whole dataset on NYT bestseller) 
# - book = positional (integer) reference of book selected as solution in Quiz
# - options = list of 4 possible answers
# - solution = string that contains right answer
# - score = previous score of user
# - kind = type of question to be asked: 1: about author, 2: about year, 3: How long on NYT list and 4: about title

def ask_Q(lst, book, options, solution, score, kind):  
    # reschuffle randomly the position of answer, otherwise right answer is always A
    choices = random.sample(options, len(options))

    # ask for author name based on argument "kind"
    if kind == 1:
        print("Who is the author of", '"'+(lst[book][0]).title()+'"?', "\nA:", choices[0], "\nB:", choices[1], "\nC:", choices[2], "\nD:",choices[3], "\nE: Answer E","\nEnter A, B, C, D or E.")
        
    # ask for published year based on argument "kind"
    elif kind == 2:
        print("In which year was", '"'+(lst[book][0]).title()+'" on the bestseller list?', "\nA:", choices[0], "\nB:", choices[1], "\nC:", choices[2], "\nD:",choices[3], "\nE: Answer E","\nEnter A, B, C, D or E.")
    
    # ask for duration on NYT list based on argument "kind"
    elif kind == 3:
        print("For how many weeks was", '"'+(lst[book][0]).title()+'" on the bestseller list?', "\nA:", choices[0], "\nB:", choices[1], "\nC:", choices[2], "\nD:",choices[3], "\nE: Answer E","\nEnter A, B, C, D or E.")
        
    # ask for book title based on argument "kind"
    elif kind == 4:
        print("Which book is written by", lst[book][2]+"?", "\nA:", choices[0].title(), "\nB:", choices[1].title(), "\nC:", choices[2].title(), "\nD:",choices[3].title(), "\nE: Answer E","\nEnter A, B, C, D or E.")
    
    # if kind is none of this above, does not ask anyq question (cannot normally occur)
    else: None

    # prompt player for an answer
    answer = input("Answer:").upper()
    
    # if answer is one of the 4 first, evaluate answer using predefined "new_score" functions
    if answer in ['A', 'B', 'C', 'D']:
        score = new_score(choices, answer, solution, score, coef = 1)
        return score
    
    # if answer E is selected, we will help user by remove one possible answer but decrease the points awarded by (1-coef)
    elif answer == 'E':
        print("Nice try! I will help you and remove one wrong answer but fewer points will be awarded if your answer is correct.\n")
        
        # remove one wrong option and shuffle order
        choices2 = random.sample(options[:3], (len(options)-1))
        
        # default answer for while loop
        answer2 = 'E'
        
        # while loop to keep prompting user as loop as answer is not A, B or C
        while answer2 not in ['A', 'B', 'C']:
            
            # Again depending on "kind", ask different questions
            if kind == 1:
                print("Who is the author of", '"'+(lst[book][0]).title()+'"?', "\nA:", choices2[0], "\nB:", choices2[1], "\nC:", choices2[2],"\nEnter A, B, or C.")
            elif kind == 2:
                print("In which year was", '"'+(lst[book][0]).title()+'" on the bestseller list?', "\nA:", choices2[0], "\nB:", choices2[1], "\nC:", choices2[2],"\nEnter A, B, or C.")
            elif kind == 3:
                print("For how many weeks was", '"'+(lst[book][0]).title()+'" on the bestseller list?', "\nA:", choices2[0], "\nB:", choices2[1], "\nC:", choices2[2],"\nEnter A, B, or C.")
            elif kind == 4:
                print("Which book is written by", lst[book][2]+"?", "\nA:", choices2[0].title(), "\nB:", choices2[1].title(), "\nC:", choices2[2].title(),"\nEnter A, B, or C.")
            else: None
                
            # prompt player for answer
            answer2 = input("Answer:").upper()
            
            # if answer does not belong to possible range, error message + ask again the question
            if answer2 not in ['A', 'B', 'C']: print("ERROR: Input out of accepted range, try again!\n")
            
        # if answer possibility valid, evaluate answer and change score
        score = new_score(choices2, answer2, solution, score, coef = 0.6) 
        return score

    # if answer is not one from A to E, ask again the original question
    else:
        print("ERROR: Input outside of accepted range. Please, try again!\n")
        return ask_Q(lst, book, options, solution, score, kind)

In [15]:
### Final Quiz ### 
# Function for QUIZ, arguments: the dataset used for the quiz and the number of rounds played
def quiz(lst = NYT_bestsellers, rounds = 5):
    # A while loop to start a new round of the quiz until the user wants to stop
    game_on = True
    while game_on:
        print('\n\033[1m' + 'Welcome to the book quiz!' +'\033[0m')
        score = 0 # Inital score of the player
        
        # While loop for every round of the quiz
        for question in range(rounds):
            # print number of question
            
            print('\n\033[1m' + 'Question '+ str(question+1) +' out of ', str(rounds) + '\033[0m')
            
            # Selection which type of question will be asked (author, year, time on list, title) at random 
            question_type = random.randint(1,4)
            
            # Depending on the selection a different if statement is fulfilled, 1 is for author questions 
            if question_type == 1:
                # A book is selected at random to quiz the user on
                book = random.randint(0,len(lst))
                # The correct solution is extracted from the data based on chosen book and question type    
                solution = lst[book][2]
                
                # Three random books are chosen for wrong answer options, we make sure they are all different from each other and the solution (issue here!!!)
                wrong_position_1 = random.choice([i for i in range(len(lst)) if lst[i][2] != lst[book][2]])
                wrong_1 = lst[wrong_position_1][2]
                wrong_position_2 = random.choice([i for i in range(len(lst)) if lst[i][2] != lst[book][2] and lst[i][2] != lst[wrong_position_1][2]])
                wrong_2 = lst[wrong_position_2][2]
                wrong_position_3 = random.choice([i for i in range(len(lst)) if lst[i][2] != lst[book][2] and lst[i][2] != lst[wrong_position_1][2] and lst[i][2] != lst[wrong_position_2][2]])
                wrong_3 = lst[wrong_position_3][2]
                # We create a list, add all answer options and then suffle them around         
                options = [solution, wrong_1, wrong_2, wrong_3]
                
                # compute score given all arguments
                score = ask_Q(lst, book, options, solution, score, 1)
            
            # If 2 was randomly generated, this indicates year questions
            elif question_type == 2: 
                # A book is selected at random to quiz the user on
                book = random.randint(0,len(lst))
                # The correct solution is extracted from the data based on chosen book and question type    
                solution = (lst[book][1]).year
                
                # Three random books are chosen for wrong answer options, we make sure they are all different from each other and the solution (issue here!!!)
                wrong_position_1 = random.choice([i for i in range(len(lst)) if (lst[i][1]).year != (lst[book][1]).year])
                wrong_1 = (lst[wrong_position_1][1]).year
                wrong_position_2 = random.choice([i for i in range(len(lst)) if (lst[i][1]).year != (lst[book][1]).year and (lst[i][1]).year != (lst[wrong_position_1][1]).year])
                wrong_2 = (lst[wrong_position_2][1]).year
                wrong_position_3 = random.choice([i for i in range(len(lst)) if (lst[i][1]).year != (lst[book][1]).year and (lst[i][1]).year != (lst[wrong_position_1][1]).year and (lst[i][1]).year != (lst[wrong_position_2][1]).year])
                wrong_3 = (lst[wrong_position_3][1]).year
                # We create a list, add all answer options and then suffle them around         
                options = [solution, wrong_1, wrong_2, wrong_3]
                
                # compute score given all arguments
                score = ask_Q(lst, book, options, solution, score, 2) 
                
            # If 3 was randomly generated, this indicates time on bestsellerlist questions
            elif question_type == 3: 
                # A book is selected at random to quiz the user on
                book = random.randint(0,len(lst))
                # The correct solution is extracted from the data based on chosen book and question type    
                solution = lst[book][5]
                
                # Three random books are chosen for wrong answer options, we make sure they are all different from each other and the solution (issue here!!!)
                wrong_position_1 = random.choice([i for i in range(len(lst)) if lst[i][5] != lst[book][5]])
                wrong_1 = lst[wrong_position_1][5]
                wrong_position_2 = random.choice([i for i in range(len(lst)) if lst[i][5] != lst[book][5] and lst[i][5] != lst[wrong_position_1][5]])
                wrong_2 = lst[wrong_position_2][5]
                wrong_position_3 = random.choice([i for i in range(len(lst)) if lst[i][5] != lst[book][5] and lst[i][5] != lst[wrong_position_1][5] and lst[i][5] != lst[wrong_position_2][5]])
                wrong_3 = lst[wrong_position_3][5]
                # We create a list, add all answer options and then suffle them around         
                options = [solution, wrong_1, wrong_2, wrong_3]
                
                # compute score given all arguments
                score = ask_Q(lst, book, options, solution, score, 3)
                    
            # If 4 is randomly generated, questions about which book is by a certain author are asked
            else:
                # A book is selected at random to quiz the user on
                book = random.randint(0,len(lst))
                # The correct solution is extracted from the data based on chosen book and question type    
                solution = lst[book][0]
                
                # Three random books are chosen for wrong answer options, we make sure they are all different from each other and the solution (issue here!!!)
                wrong_position_1 = random.choice([i for i in range(len(lst)) if lst[i][0] != lst[book][0]])
                wrong_1 = lst[wrong_position_1][0]
                wrong_position_2 = random.choice([i for i in range(len(lst)) if lst[i][0] != lst[book][0] and lst[i][0] != lst[wrong_position_1][0]])
                wrong_2 = lst[wrong_position_2][0]
                wrong_position_3 = random.choice([i for i in range(len(lst)) if lst[i][0] != lst[book][0] and lst[i][0] != lst[wrong_position_1][0] and lst[i][0] != lst[wrong_position_2][0]])
                wrong_3 = lst[wrong_position_3][0]
                # We create a list, add all answer options and then suffle them around         
                options = [solution, wrong_1, wrong_2, wrong_3]
                
                # compute score given all arguments
                score = ask_Q(lst, book ,options, solution, score, 4)
                
        # End of game, ask if user wants to play again, if yes, while loop continues, else loop stops
        print("\nEND! Final score:", score)
        
        # ask question for another party
        game_on = binary_Q('\nDo you want to play another party?')

In [16]:
### Final Quiz ### 
# Function for QUIZ, arguments: the dataset used for the quiz and the number of rounds played
def quiz(lst = NYT_bestsellers, rounds = 5):
    # A while loop to start a new round of the quiz until the user wants to stop
    game_on = True
    while game_on:
        print('\n\033[1m' + 'Welcome to the book quiz!' +'\033[0m')
        score = 0 # Inital score of the player
        
        # While loop for every round of the quiz
        for question in range(rounds):
            # print number of question
            
            print('\n\033[1m' + 'Question '+ str(question+1) +' out of ', str(rounds) + '\033[0m')
            
            # Selection which type of question will be asked (author, year, time on list, title) at random 
            question_type = random.randint(1,4)
            
            # Depending on the selection a different if statement is fulfilled, 1 is for author questions 
            if question_type == 1:
                # A book is selected at random to quiz the user on
                book = random.randint(0,len(lst))
                # The correct solution is extracted from the data based on chosen book and question type    
                solution = lst[book][2]
                
                # Three random books are chosen for wrong answer options, we make sure they are all different from each other and the solution (issue here!!!)
                wrong_position_1 = random.choice([i for i in range(len(lst)) if lst[i][2] != lst[book][2]])
                wrong_1 = lst[wrong_position_1][2]
                wrong_position_2 = random.choice([i for i in range(len(lst)) if lst[i][2] != lst[book][2] and lst[i][2] != lst[wrong_position_1][2]])
                wrong_2 = lst[wrong_position_2][2]
                wrong_position_3 = random.choice([i for i in range(len(lst)) if lst[i][2] != lst[book][2] and lst[i][2] != lst[wrong_position_1][2] and lst[i][2] != lst[wrong_position_2][2]])
                wrong_3 = lst[wrong_position_3][2]
                # We create a list, add all answer options and then suffle them around         
                options = [solution, wrong_1, wrong_2, wrong_3]
                
                # compute score given all arguments
                score = ask_Q(lst, book, options, solution, score, 1)
            
            # If 2 was randomly generated, this indicates year questions
            elif question_type == 2: 
                # A book is selected at random to quiz the user on
                book = random.randint(0,len(lst))
                # The correct solution is extracted from the data based on chosen book and question type    
                solution = (lst[book][1]).year
                
                # Three random books are chosen for wrong answer options, we make sure they are all different from each other and the solution (issue here!!!)
                wrong_position_1 = random.choice([i for i in range(len(lst)) if (lst[i][1]).year != (lst[book][1]).year])
                wrong_1 = (lst[wrong_position_1][1]).year
                wrong_position_2 = random.choice([i for i in range(len(lst)) if (lst[i][1]).year != (lst[book][1]).year and (lst[i][1]).year != (lst[wrong_position_1][1]).year])
                wrong_2 = (lst[wrong_position_2][1]).year
                wrong_position_3 = random.choice([i for i in range(len(lst)) if (lst[i][1]).year != (lst[book][1]).year and (lst[i][1]).year != (lst[wrong_position_1][1]).year and (lst[i][1]).year != (lst[wrong_position_2][1]).year])
                wrong_3 = (lst[wrong_position_3][1]).year
                # We create a list, add all answer options and then suffle them around         
                options = [solution, wrong_1, wrong_2, wrong_3]
                
                # compute score given all arguments
                score = ask_Q(lst, book, options, solution, score, 2) 
                
            # If 3 was randomly generated, this indicates time on bestsellerlist questions
            elif question_type == 3: 
                # A book is selected at random to quiz the user on
                book = random.randint(0,len(lst))
                # The correct solution is extracted from the data based on chosen book and question type    
                solution = lst[book][5]
                
                # Three random books are chosen for wrong answer options, we make sure they are all different from each other and the solution (issue here!!!)
                wrong_position_1 = random.choice([i for i in range(len(lst)) if lst[i][5] != lst[book][5]])
                wrong_1 = lst[wrong_position_1][5]
                wrong_position_2 = random.choice([i for i in range(len(lst)) if lst[i][5] != lst[book][5] and lst[i][5] != lst[wrong_position_1][5]])
                wrong_2 = lst[wrong_position_2][5]
                wrong_position_3 = random.choice([i for i in range(len(lst)) if lst[i][5] != lst[book][5] and lst[i][5] != lst[wrong_position_1][5] and lst[i][5] != lst[wrong_position_2][5]])
                wrong_3 = lst[wrong_position_3][5]
                # We create a list, add all answer options and then suffle them around         
                options = [solution, wrong_1, wrong_2, wrong_3]
                
                # compute score given all arguments
                score = ask_Q(lst, book, options, solution, score, 3)
                    
            # If 4 is randomly generated, questions about which book is by a certain author are asked
            else:
                # A book is selected at random to quiz the user on
                book = random.randint(0,len(lst))
                # The correct solution is extracted from the data based on chosen book and question type    
                solution = lst[book][0]
                
                # Three random books are chosen for wrong answer options, we make sure they are all different from each other and the solution (issue here!!!)
                wrong_position_1 = random.choice([i for i in range(len(lst)) if lst[i][0] != lst[book][0]])
                wrong_1 = lst[wrong_position_1][0]
                wrong_position_2 = random.choice([i for i in range(len(lst)) if lst[i][0] != lst[book][0] and lst[i][0] != lst[wrong_position_1][0]])
                wrong_2 = lst[wrong_position_2][0]
                wrong_position_3 = random.choice([i for i in range(len(lst)) if lst[i][0] != lst[book][0] and lst[i][0] != lst[wrong_position_1][0] and lst[i][0] != lst[wrong_position_2][0]])
                wrong_3 = lst[wrong_position_3][0]
                # We create a list, add all answer options and then suffle them around         
                options = [solution, wrong_1, wrong_2, wrong_3]
                
                # compute score given all arguments
                score = ask_Q(lst, book ,options, solution, score, 4)
                
        # End of game, ask if user wants to play again, if yes, while loop continues, else loop stops
        print("\nEND! Final score:", score)
        
        # ask question for another party
        game_on = binary_Q('\nDo you want to play another party?')

### Check function
Uncomment code line to try quiz

In [17]:
#quiz(rounds = 6)

# 05 Final Project – NYT Bestsellers
##  Search Engine & QUIZ
Prompt user and call all previous functions to present project

In [19]:
# define some text to be used later in variables
intro_txt1 ="""We build a search engine and a little quiz that offers the following options:
- Search the NYT bestseller by title, author, date, week on list or price range
- Advanced seach where you can filter the list by multiple criteria
- Quiz with a few multiple choice questions"""

intro_txt2 = """\nWhat would you like to do now?
- Search by Criterion (1)\n- Advanced Search (2)\n- Quiz (3)"""

txt_choice = """\nWhat would you like to do now?
- Search by Criterion  (type: 1)
- Advanced Search      (type: 2)
- Quiz                 (type: 3)
Your choice: """

expl_txt = """It allows you to filter the dataset by the following criteria;
- book title                                (type: 'title')
- author name(s)                            (type: 'author')
- published date of the NYT bestseller list (type: 'date')
- number of weeks on NYTB list              (type: 'week')
- book price range                          (type: 'price')"""

# define function that prompt user for 1, 2 or 3 and check for invalid inputs
def prompt_choice():
    choice = input(txt_choice)
    #choice = input('type: 1 to search by criterion, 2 for the advanced search or 3 for the quiz: ')
    if choice in ['1', '2', '3']:
        return choice
    else:
        print("Input does not fit requirements, please enter 1, 2 or 3!")
        return prompt_choice()

    
### Greeting and explantions
print('\033[1m' + 'Welcome to our project about the NYT bestseller list!\n' + '\033[0m')
print(intro_txt1)

# keep offering the possibility to search the NYT bestseller list or quiz as long as user does not want to quit
cont = True
while cont:
    # present three possibilities
    #print(intro_txt2)

    # Ask user for his choice using predefined function 
    choice = prompt_choice()
    
    # run basic search if user selects 1
    if choice == '1':
        #print(expl_txt)
        result = search_select()
        res_search(result)
        display_n_book(result)

    # run advanced search if user selects 2
    elif choice == '2':
        adv_search()
        
    # run advanced search if user selects 3
    else:
        quiz(rounds = 7)
    
    print("\nOperation done!")
    
    # Ask user if want to continue after finished with one of the above applications
    cont = binary_Q('Do you want to continue using the application? ')

Welcome to our project about the NYT bestseller list!

We build a search engine and a little quiz that offers the following options:
- Search the NYT bestseller by title, author, date, week on list or price range
- Advanced seach where you can filter the list by multiple criteria
- Quiz with a few multiple choice questions

What would you like to do now?
- Search by Criterion  (type: 1)
- Advanced Search      (type: 2)
- Quiz                 (type: 3)
Your choice: 1

Possible search criteria:
- book title                                (type: title)
- author name(s)                            (type: author)
- published date of the NYT bestseller list (type: date)
- number of weeks on the NYTB list          (type: week)
- book price range                          (type: price)
Which criterion do you want to search by? author
Please enter the author name(s) you want to search for.
Author(s): rowling

Results of your search:
- Harry Potter by JK Rowling | First time on NYTB list: 2010-11-